In [12]:
import glob as g
import pyke
import scipy
from photutils import detect_threshold, detect_sources, deblend_sources
import numpy as np
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.convolution import Gaussian2DKernel
from k2sc.standalone import k2sc_lc
from astropy.io import fits
import lightkurve as lk
from sklearn.cluster import DBSCAN

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [22]:
class star:
    
    def __init__(self, ID):
        self.ID = ID
        self.fits_image_filename=self.ID[:-5]+"-kepextract.fits"
        
    def kepextract(self):
        pyke.kepextract(self.ID,maskfile='ALL',psfcentroid=True,overwrite=True)
        with fits.open(self.fits_image_filename) as hdul:
            data=hdul[1].data
        self.psfc1=data['PSF_CENTR1']
        self.psfc2=data['PSF_CENTR2']
        
        newpsfcP=[]
        for index, line in enumerate(self.psfc1): newpsfcP.append([self.psfc1[index],self.psfc2[index]])
        self.newpsfc=np.asarray(newpsfcP)

    def tpfs(self):
        self.tpf=lk.targetpixelfile.KeplerTargetPixelFile(self.ID)
        
    def clustering(self):
        db = DBSCAN(eps=0.3, min_samples=10).fit(self.newpsfc)
        self.core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        self.core_samples_mask[db.core_sample_indices_] = True
        
    def apcalc(self):
        
        self.countergrid_all=np.asarray([[0]*len(self.tpf.flux[0][0]) for n in range(len(self.tpf.flux[0]))])
        
        for i in range(len(self.tpf.flux)):
            if i in np.where(self.core_samples_mask==True)[0]:
                tpfdata=self.tpf.flux[i]

                threshold = detect_threshold(tpfdata, snr=1.8)
                sigma = 3.0 * gaussian_fwhm_to_sigma    # FWHM = 3.
                kernel = Gaussian2DKernel(sigma, x_size=1, y_size=1)
                kernel.normalize()
                segm = detect_sources(tpfdata, threshold, npixels=1, filter_kernel=kernel)
                segm_deblend = deblend_sources(tpfdata, segm, npixels=5, filter_kernel=kernel)

                apx=[];apy=[]
                for x in range(len(segm.data)):
                    for y in range(len(segm.data[0])):
                        if segm.data[x][y]!=0:
                            apx.append(x)
                            apy.append(y)

                self.countergrid_all += segm.data

        limitc=(self.countergrid_all>len(self.tpf.flux)/2.5)
        
        self.labelgrid, self.num_features = scipy.ndimage.measurements.label(limitc)      
        
        
    def getlargestap(self):
        
        sizeoffeature=0
        self.largestfeature=0
        
        self.featuresizelist=[]
        
        for i in range(self.num_features):
        
            sizeofthis=len(np.where(self.labelgrid == (i+1))[0])
            
            self.featuresizelist.append(sizeofthis)
            
            if sizeofthis>sizeoffeature:
                
                sizeoffeature=sizeofthis
                self.largestfeature=(i+1)

        
    def make_lc(self):
        
        lc = self.tpf.to_lightcurve(aperture_mask=(self.labelgrid==(self.largestfeature)).astype(bool)) # load some data either as a tpf or just straight up as a lightcurve
        lc.primary_header = self.tpf.hdu[0].header
        lc.data_header = self.tpf.hdu[1].header
        lc.pos_corr1 = self.tpf.hdu[1].data['POS_CORR1'][self.tpf.quality_mask]
        lc.pos_corr2 = self.tpf.hdu[1].data['POS_CORR2'][self.tpf.quality_mask]
        lc.__class__ = k2sc_lc
        lc.k2sc()
        self.ourlc=lc
        
    def writeout(self):
        
        fout=open(self.ID+"OOPoutII",'w')
        fout.write(str(self.ID)+"\n")
        fout.write("size_of_found_features:\n")
        for each in self.featuresizelist: fout.write(str(each)+" ")
        fout.write("\n")
        fout.write("lc.time lc.corr_flux\n")
        for index, each in enumerate(self.ourlc.time):
            fout.write(str(each)+" "+str(self.ourlc.corr_flux[index])+"\n")
        fout.close()

In [23]:
OURSTARS=g.glob('/home/pal/konkoly/automaticaperture/sourcefits/ktwo*targ.fits')

In [24]:
for EACH in OURSTARS:
    ourstar=star(ID=EACH)
    ourstar.kepextract()
    ourstar.tpfs()
    ourstar.clustering()
    ourstar.apcalc()
    ourstar.getlargestap()
    ourstar.make_lc()
    ourstar.writeout()

PSF centroiding: 100%|██████████| 3561/3561 [00:10<00:00, 339.23it/s]
/home/pal/.local/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:165: RuntimeWarning: invalid value encountered in greater
  _filtered_data.mask |= _filtered_data > max_value
/home/pal/.local/lib/python3.6/site-packages/astropy/stats/sigma_clipping.py:166: RuntimeWarning: invalid value encountered in less
  _filtered_data.mask |= _filtered_data < min_value
/home/pal/.local/lib/python3.6/site-packages/photutils/segmentation/detect.py:128: RuntimeWarning: invalid value encountered in greater
  check_normalization=True) > threshold)


Using default splits [2344] for campaign 5
Starting initial outlier detection
  Flagged 0 ( 0.0%) outliers.
Starting Lomb-Scargle period search
  Using SqrExp position kernel
  Found periodicity p =    0.58 (fap 0.0000e+00 < 1e-50), will use a quasiperiodic kernel
Starting global hyperparameter optimisation using DE
  DE iteration %3i -ln(L) %4.1f 0 1228.7539455813248
  DE iteration %3i -ln(L) %4.1f 1 1228.7539455813248
  DE iteration %3i -ln(L) %4.1f 2 -839.1827263852257
  DE iteration %3i -ln(L) %4.1f 3 -839.1827263852257
  DE iteration %3i -ln(L) %4.1f 4 -938.3154560487983
  DE iteration %3i -ln(L) %4.1f 5 -1087.6615982525293
  DE iteration %3i -ln(L) %4.1f 6 -1087.6615982525293
  DE iteration %3i -ln(L) %4.1f 7 -1087.6615982525293
  DE iteration %3i -ln(L) %4.1f 8 -1087.6615982525293
  DE iteration %3i -ln(L) %4.1f 9 -1087.6615982525293
  DE iteration %3i -ln(L) %4.1f 10 -1115.8910053855836
  DE iteration %3i -ln(L) %4.1f 11 -1115.8910053855836
  DE iteration %3i -ln(L) %4.1f 12 -1

/home/pal/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: ResourceWarning: unclosed file <_io.FileIO name='/home/pal/konkoly/automaticaperture/sourcefits/ktwo212412494-c06_lpd-targ.fits' mode='rb' closefd=True>
  
Aperture photometry:   0%|          | 0/3561 [00:00<?, ?it/s]

  CDPP - raw - %6.3f 93650.00587904525
  CDPP - position component removed - %6.3f 92759.60528357327
  CDPP - full reduction - %6.3f 2315.5004753152775
Detrending time %6.3f 343.50878047943115


PSF centroiding: 100%|██████████| 3561/3561 [00:10<00:00, 339.67it/s]
/home/pal/.local/lib/python3.6/site-packages/numpy/ma/core.py:3042: ResourceWarning: unclosed file <_io.FileIO name='/home/pal/konkoly/automaticaperture/sourcefits/ktwo212412494-c06_lpd-targ.fits' mode='rb' closefd=True>
  m = reduce(mask_or, [getmaskarray(arg) for arg in input_args])


Using default splits [2344] for campaign 5
Starting initial outlier detection
  Flagged 0 ( 0.0%) outliers.
Starting Lomb-Scargle period search
  Using SqrExp position kernel
  Found periodicity p =    0.32 (fap 0.0000e+00 < 1e-50), will use a quasiperiodic kernel
Starting global hyperparameter optimisation using DE
  DE iteration %3i -ln(L) %4.1f 0 847.8419818980956
  DE iteration %3i -ln(L) %4.1f 1 -610.5782340315551
  DE iteration %3i -ln(L) %4.1f 2 -1237.460404720246
  DE iteration %3i -ln(L) %4.1f 3 -1237.460404720246
  DE iteration %3i -ln(L) %4.1f 4 -1494.555573069934
  DE iteration %3i -ln(L) %4.1f 5 -1494.555573069934
  DE iteration %3i -ln(L) %4.1f 6 -1520.7159326543247
  DE iteration %3i -ln(L) %4.1f 7 -1534.511801482377
  DE iteration %3i -ln(L) %4.1f 8 -1678.25414487034
  DE iteration %3i -ln(L) %4.1f 9 -1678.25414487034
  DE iteration %3i -ln(L) %4.1f 10 -1678.25414487034
  DE iteration %3i -ln(L) %4.1f 11 -1678.25414487034
  DE iteration %3i -ln(L) %4.1f 12 -1678.2541448

KeyboardInterrupt: 